### Random Forest Model por Failure Machine Detection
This script will load a set of tabular data with a target feature to perform supervices training of a electrical generator and predict a failure.

-Jonathan Espín Martin 00339306
- Master Degree

Get the dataframe (tabular)

In [ ]:
#data frame definition based on files
import pandas as pd
import os
exportPath = r"/home/jespinm/power_gen_data/Export10Sec"
files_data = os.listdir(exportPath)
powerdata_dfs = []
for fd in sorted(files_data):
  if fd.endswith(".csv"):
    if fd.endswith("_01.csv"):
      print(fd)
    powerdata_dfs.append(pd.read_csv(f'{exportPath}/%s' % fd,engine='pyarrow', quotechar='"',doublequote=True))
power_df = pd.concat(powerdata_dfs, ignore_index = True)
power_df.t_stamp = pd.to_datetime(power_df.t_stamp)
power_df = power_df.sort_values(by='t_stamp', ascending=True)
power_df.set_index("t_stamp", inplace = True)

#fill nans and clip to 0 (little eda)
power_df = power_df.fillna(0)
power_df = power_df.clip(lower=0)
power_df

2019_05_01.csv
2019_06_01.csv
2019_07_01.csv
2019_08_01.csv
2019_09_01.csv
2019_10_01.csv
2019_11_01.csv
2019_12_01.csv
2020_01_01.csv
2020_02_01.csv
2020_03_01.csv
2020_04_01.csv
2020_05_01.csv
2020_06_01.csv
2020_07_01.csv
2020_08_01.csv
2020_09_01.csv
2020_10_01.csv
2020_11_01.csv
2020_12_01.csv
2021_01_01.csv
2021_02_01.csv
2021_03_01.csv
2021_04_01.csv
2021_05_01.csv
2021_06_01.csv
2021_07_01.csv
2021_08_01.csv
2021_09_01.csv
2021_10_01.csv
2021_11_01.csv


,cfcfe6/g6actuatorout,cfcfe6/g6aparentpowerpmu,cfcfe6/g6avgcyltmp,cfcfe6/g6enginestop,cfcfe6/g6frequencyhzeu,cfcfe6/g6fueloilcomsum,cfcfe6/g6fueloilinflow,cfcfe6/g6fueloiloutflow,cfcfe6/g6hzcv,cfcfe6/g6kwsp,...,cfcfe6/g6te708,cfcfe6/g6te709,cfcfe6/g6te710,cfcfe6/g6te711,cfcfe6/g6te801eu,cfcfe6/g6te802eu,cfcfe6/g6te803eu,cfcfe6/g6te804eu,cfcfe6/g6te805eu,cfcfe6/g6failure
t_stamp,,,,,,,,,,,,,,,,,,,,,
2019-05-01 00:00:00,0.000000,6489.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
2019-05-01 00:00:10,0.000000,6498.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
2019-05-01 00:00:20,0.000000,6480.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
2019-05-01 00:00:30,0.000000,6500.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
2019-05-01 00:00:40,0.000000,6487.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-30 23:59:10,3.227802,59.748323,42.584731,0,0.285149,7.653473,4.879628,6.416220,1.683695,6694.599590,...,0.0,0.0,0.0,0.0,81.863647,82.829318,91.286142,79.194735,82.513632,0
2021-11-30 23:59:20,3.227814,59.748557,42.584773,0,0.285150,7.653503,4.879646,6.416245,1.683701,6694.599044,...,0.0,0.0,0.0,0.0,81.863649,82.829323,91.286138,79.194736,82.513634,0
2021-11-30 23:59:30,3.227826,59.748790,42.584814,0,0.285151,7.653532,4.879665,6.416270,1.683708,6694.598497,...,0.0,0.0,0.0,0.0,81.863652,82.829327,91.286134,79.194738,82.513636,0


Get some information of the dataframe

In [ ]:
#get some info from the dataset
power_df.info()
num_rows, num_columns = power_df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8164800 entries, 2019-05-01 00:00:00 to 2021-11-30 23:59:50
Data columns (total 89 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   cfcfe6/g6actuatorout       float64
 1   cfcfe6/g6aparentpowerpmu   float64
 2   cfcfe6/g6avgcyltmp         float64
 3   cfcfe6/g6enginestop        int64  
 4   cfcfe6/g6frequencyhzeu     float64
 5   cfcfe6/g6fueloilcomsum     float64
 6   cfcfe6/g6fueloilinflow     float64
 7   cfcfe6/g6fueloiloutflow    float64
 8   cfcfe6/g6hzcv              float64
 9   cfcfe6/g6kwsp              float64
 10  cfcfe6/g6loadkweu          float64
 11  cfcfe6/g6pf                float64
 12  cfcfe6/g6pmuil1            float64
 13  cfcfe6/g6pmuil2            float64
 14  cfcfe6/g6pmuil3            float64
 15  cfcfe6/g6pmuu12            float64
 16  cfcfe6/g6pmuu23            float64
 17  cfcfe6/g6pmuu31            float64
 18  cfcfe6/g6presssp           float64
 19  cfcfe6/g6

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import graphviz
import altair as alt
import joblib
from sklearn.metrics import make_scorer, recall_score, fbeta_score

Prepare the data for training, making train test splits, variance feature validation.

In [ ]:
#set seed
seed = 11
data = power_df

#separe the train set and target vector
X = power_df.drop('cfcfe6/g6failure', axis=1)
y = power_df['cfcfe6/g6failure']

#check for missing or infinite values in the data
missing_values = X.isna().sum().sum()
non_finite_values = (~np.isfinite(X)).sum().sum()
print(f"Missing values: {missing_values}")
print(f"Non-finite values: {non_finite_values}")

#replace or drop missing/non-finite values if any
if missing_values > 0:
    X = X.fillna(X.mean())
if non_finite_values > 0:
    X = X.replace([np.inf, -np.inf], np.nan).dropna()

#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
selector = VarianceThreshold(threshold=0.0)
X_train_selected = selector.fit_transform(X_train)
selected_features = selector.get_support(indices=True)
selected_feature_names = X_train.columns[selected_features]
X_train = pd.DataFrame(X_train_selected, columns=selected_feature_names)
X_test_selected = selector.transform(X_test)
X_test = pd.DataFrame(X_test_selected, columns=selected_feature_names)

Missing values: 0
Non-finite values: 0


Train using a Pipeline for hyper parameters testing and keeping best results also save the model and some features information

In [ ]:
 import os
import joblib
import pandas as pd
import threading
import time
import psutil
import os
from itertools import product

#training pipeline, balance, kbest, normalize, pca and classifier
pipeline = Pipeline([
    ('smote', SMOTE(random_state=seed)),
    ('selector', SelectKBest(score_func=f_classif, k=20)),
    ('scaler', MinMaxScaler()),
    ('pca', PCA()),
    ('clf', RandomForestClassifier(random_state=seed))
])

#grid params
param_grid = {
    'smote__sampling_strategy': [0.8],
    'smote__k_neighbors': [6, 7],
    'clf__max_depth': [5, 8],
    'clf__n_estimators': np.unique(np.logspace(np.log10(70), np.log10(150), num=3, dtype=int)).tolist(),
    'pca__n_components': [0.95, None] #.95 variance or none
}

#random search grid search hiperparameters
scoring = {'recall': make_scorer(recall_score),'f2': make_scorer(fbeta_score, beta=2)}
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=len(list(product(*param_grid.values()))),
    scoring=scoring,
    refit='f2',
    cv=cv,
    n_jobs=15,
    verbose=10,
    random_state=seed,
    error_score='raise',
    pre_dispatch='2*n_jobs'
)

#start monitor time
start_time = time.time()

#train
random_search.fit(X_train, y_train)

#endmontor monitoring time
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution Time: {execution_time:.2f} seconds")

#set dir to save
model_dir = '/home/jespinm/models/RandomForest03'
os.makedirs(model_dir, exist_ok=True)

#save model
model_path = os.path.join(model_dir, 'best_model.pkl')
joblib.dump(random_search.best_estimator_, model_path)

#save feature importance
feature_names_filename = os.path.join(model_dir, 'feature_names.pkl')
joblib.dump(X_train.columns.tolist(), feature_names_filename)

#results extractore
results = {
    "best_params": random_search.best_params_,
    "best_score": random_search.best_score_,
    "cv_results": random_search.cv_results_,
}

#random search results serial
results_path = os.path.join(model_dir, 'random_search_results.pkl')
joblib.dump(results, results_path)

#random search csc
cv_results_df = pd.DataFrame(random_search.cv_results_)
csv_path = os.path.join(model_dir, 'random_search_results.csv')
cv_results_df.to_csv(csv_path, index=False)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
Execution Time: 9368.14 seconds


--------------------------------------------------------------------------------
This separation represents the use of the trained model on other devices

In [ ]:
#load model and other features
import os
import joblib
import pandas as pd
model_dir = '/home/jespinm/models/RandomForest03'
loaded_model = joblib.load(os.path.join(model_dir, 'best_model.pkl'))
loaded_feature_names = joblib.load(os.path.join(model_dir, 'feature_names.pkl'))
loaded_results = joblib.load(os.path.join(model_dir, 'random_search_results.pkl'))
loaded_csv_results = pd.read_csv(os.path.join(model_dir, 'random_search_results.csv'))

print("Model, results, and feature names loaded successfully.")

Model, results, and feature names loaded successfully.


Display some information of the model inside the pipeline

In [ ]:
#print some model information
best_params = loaded_results['best_params']
best_max_depth = best_params['clf__max_depth']
best_n_estimators = best_params['clf__n_estimators']
best_sampling_strategy = best_params['smote__sampling_strategy']
best_k_neighbors = best_params['smote__k_neighbors']

print('Best max_depth:', best_max_depth)
print('Best n_estimators:', best_n_estimators)
print('Best SMOTE sampling_strategy:', best_sampling_strategy)
print('Best SMOTE k_neighbors:', best_k_neighbors)

pipeline = loaded_model
pipeline.set_params(**best_params)
pipeline

Best max_depth: 8
Best n_estimators: 70
Best SMOTE sampling_strategy: 0.8
Best SMOTE k_neighbors: 7


Pipeline(steps=[('smote',
                 SMOTE(k_neighbors=7, random_state=11, sampling_strategy=0.8)),
                ('selector', SelectKBest(k=20)), ('scaler', MinMaxScaler()),
                ('pca', PCA()),
                ('clf',
                 RandomForestClassifier(max_depth=8, n_estimators=70,
                                        random_state=11))])

Save the model with the best params update

In [ ]:
#save model pepile with best params
model_filename = os.path.join(model_dir, 'trained_random_forest_pipeline_model.pkl')
joblib.dump(pipeline, model_filename)
print(f"Model updated saved to {model_filename}")

Model updated saved to /home/jespinm/models/RandomForest03/trained_random_forest_pipeline_model.pkl


evaluate the model

In [ ]:
from sklearn.metrics import fbeta_score, confusion_matrix, classification_report

#function to evaluate a pipeline using F2 score
def evaluate_pipeline(pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    #calculate F2 score (beta=2)
    f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return report, f2, cm

#evaluate the best model on the test set using F2 score
eval_report, eval_f2, eval_cm = evaluate_pipeline(pipeline, X_train, y_train, X_test, y_test)
print(eval_report)
print('F2 Score:', eval_f2)
print('Confusion Matrix:')
print(eval_cm)

              precision    recall  f1-score   support

           0       1.00      0.94      0.97   1508219
           1       0.58      1.00      0.73    124741

    accuracy                           0.94   1632960
   macro avg       0.79      0.97      0.85   1632960
weighted avg       0.97      0.94      0.95   1632960

F2 Score: 0.945010897204909
Confusion Matrix:
[[1417214   91005]
 [    239  124502]]


Features descriptor

In [ ]:
#replace feature names with descriptions
features_reference_df = pd.read_csv("/home/jespinm/power_references/features_reference.csv", delimiter=',')
features_reference = features_reference_df.set_index('tag')['feature'].to_dict()
def get_feature_description(feature_name):
    return features_reference.get(feature_name, feature_name)
features_reference

{'cfcfe6/g6te703': 'Main bearing 3 temp.',
 'cfcfe6/g6te401eu': 'HT-water temp. jacket inlet',
 'cfcfe6/g6tc51b': 'Exh. gas temp. cyl. B1',
 'cfcfe6/g6fueloilcomsum': 'Fuel oil consumption from flow rate',
 'cfcfe6/g6pressspmin': 'Receiver press. control, sp. min. limit',
 'cfcfe6/g6pt201eu': 'Lube oil inlet pressure',
 'cfcfe6/g6te704': 'Main bearing 4 temp.',
 'cfcfe6/g6te805eu': 'Gen. ND-end bearing temp.',
 'cfcfe6/g6frequencyhzeu': 'Generator frequency',
 'cfcfe6/g6tc56b': 'Exh. gas temp. cyl. B6',
 'cfcfe6/g6te521': 'Exhaust gas temp TC B inlet',
 'cfcfe6/g6te804eu': 'Gen. D-end bearing temp.',
 'cfcfe6/g6tc59a': 'Exh. gas temp. cyl. A9',
 'cfcfe6/g6pmuu23': 'Generator main voltage U23',
 'cfcfe6/g6fueloilinflow': 'Fuel oil inlet flow',
 'cfcfe6/g6st528eu': 'Turbo B speed',
 'cfcfe6/g6pt471eu': 'LT-water pressure CAC inlet',
 'cfcfe6/g6tc52b': 'Exh. gas temp. cyl. B2',
 'cfcfe6/g6tc58b': 'Exh. gas temp. cyl. B8',
 'cfcfe6/g6te709': 'Main bearing 9 temp.',
 'cfcfe6/g6te702': 'Main

Feautes Importance selector and random tree representation

In [ ]:
#feature importance selector
model_dir = '/home/jespinm/models/RandomForest03'
estimator = pipeline.named_steps['clf'].estimators_[0]
selected_features_mask = pipeline.named_steps['selector'].get_support()
selected_feature = [feature for feature, mask in zip(loaded_feature_names,selected_features_mask) if mask]
for f in selected_feature:
    print(get_feature_description(f))
dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=[get_feature_description(feature) for feature in selected_features[0:estimator.n_features_in_]],
                           filled=True, rounded=True,
                           special_characters=True,
                           max_depth=best_max_depth)

#random forest tree in pdf
title = "Machine Failure Decision Tree"
dot_data = f"digraph Tree {{\nlabel=\"{title}\";\nlabelloc=t;\n{dot_data.split('digraph Tree {')[1]}"

#render the tree and save
graph = graphviz.Source(dot_data)
output_path = os.path.join(model_dir, "decision_tree")
graph.render(output_path, format='pdf', cleanup=True)
print("\nDecision tree saved")

Actuator position %
Generator aparent power
Generator frequency
Fuel oil consumption from flow rate
Generator active power
Generator phase current L1
Generator phase current L2
Generator phase current L3
Generator main voltage U12
Generator main voltage U23
Generator main voltage U31
Lube oil inlet pressure
Lube oil pressure TC A inlet
Lube oil pressure TC B inlet
HT-water pressure jacket inlet
LT-water pressure CAC inlet
CA pressure engine inlet
Engine speed
Turbo A speed
Turbo B speed

Decision tree saved


Feature Importance Analysis

In [ ]:
import pandas as pd
import numpy as np

#get the RandomForestClassifier from the pipeline
random_forest_model = pipeline.named_steps['clf']

#extract feature importance from the RandomForestClassifier
feature_importances = random_forest_model.feature_importances_

#get the mask of the selected features after SelectKBest
selected_feature_indices = pipeline.named_steps['selector'].get_support(indices=True)

#get the selected feature names (only the top 20 features selected by SelectKBest)
selected_feature_names = [get_feature_description(loaded_feature_names[i]) for i in selected_feature_indices]

#create a DataFrame for better visualization using only the selected features
feature_importance_df = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': feature_importances
})

#sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

#display the top 20 features
print(feature_importance_df.head(20))

#save the feature importance DataFrame as a .pkl file
importance_file = os.path.join(model_dir, 'feature_importance.pkl')
joblib.dump(feature_importance_df, importance_file)

print(f"Feature importance saved as pickle file to {importance_file}")

                                Feature  Importance
0                   Actuator position %    0.355721
6            Generator phase current L2    0.138774
1               Generator aparent power    0.119075
3   Fuel oil consumption from flow rate    0.106275
2                   Generator frequency    0.067625
4                Generator active power    0.042644
12         Lube oil pressure TC A inlet    0.040798
5            Generator phase current L1    0.019866
17                         Engine speed    0.014735
9            Generator main voltage U23    0.014434
7            Generator phase current L3    0.014271
8            Generator main voltage U12    0.012243
13         Lube oil pressure TC B inlet    0.011766
14       HT-water pressure jacket inlet    0.009375
10           Generator main voltage U31    0.008991
11              Lube oil inlet pressure    0.008630
16             CA pressure engine inlet    0.007130
15          LT-water pressure CAC inlet    0.003725
18          